In [2]:
import re
import io
import json
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
import nltk
import functools
import tokenizations
from nltk.tokenize import word_tokenize 
from nltk.tokenize import WordPunctTokenizer
import pandas as pd
import os
import glob
import os.path
import numpy as np
from pathlib import Path

# starting CoNLL conversion from annotated file(.ann)

In [3]:
def read_input_ann(a_file): # reading the brat output annotation file
    txt = open(a_file).read()       
    return txt

In [4]:
def spans(txt):
    tokenizer = WordPunctTokenizer() # works best
    tokens = tokenizer.tokenize(txt)
    offset = 0
    for token in tokens:
        offset = txt.find(token, offset)
        yield token, offset, offset+len(token)
        offset += len(token)

In [5]:
def generate_main_list(txt): # adding "O" tags to all tokens
    main_list =[]
    for token in spans(txt):
        pos_tag = nltk.pos_tag([token[0]])[0]
        main_list.append([token[1],token[0],pos_tag[1], "O"])
    return main_list  

In [6]:
def get_input_file(filename): # checking validation
    if(filename[len(filename)-4:len(filename)] != ".ann"):
        print("Invalid File")
        getInputFile()
    else:
        file = open(filename, "r")
        return file  

In [7]:
def generate_separate_tags(tokens, main_list):
    if 'R' in tokens[0]: # if it is a relationship link, ignore for now
        return main_list
    else:
        if len(tokens) > 5:
            temp = list()
            temp = tokens[4:]
            tag = tokens[1]
            start = tokens[2]
            end = tokens[3]
            first_tok = tokens[4]
            first_tag_string = "B-" + str(tag)
            rest_tag_string = "I-" + str(tag)
            flag = True
            for ann_toks in temp:
                # mapping words from entity list to main list and overlapping to create final list
                for toks in main_list: 
                    if int(toks[0]) == int(start) and flag:
                        toks[3] = first_tag_string
                        flag = False
                        break
                # changing the main list token tags within the start and end range of the annotated tokens
                    if toks[0] < int(end) and toks[0] >= int(start) and ann_toks == toks[1]:
                        toks[3] = rest_tag_string
                        break
                        
        else: # if there only is a single token annotated
            tag = tokens[1]
            start = tokens[2]
            end = tokens[3]
            first_tok = tokens[4]
            first_tag_string = "B-" + str(tag)
            for toks in main_list:
                if int(toks[0]) == int(start):
                    toks[3] = first_tag_string
                    break
    return main_list

In [8]:
def map_lists(filename, main_list): # adding respective entity tags to annotated text
    file = get_input_file(filename)
    new_lst = []
    for i in file:
        tokenizer = WordPunctTokenizer() # using the same tokenizer as the main_list to avoid indexing issues
        tokens = tokenizer.tokenize(i)
        new_lst = generate_separate_tags(tokens, main_list)
    return new_lst

In [9]:
def export_to_CoNLL(final_list): # converting to required CoNLL format
    final_df = pd.DataFrame(final_list)
    tokens = final_df[1]
    pos_labels = final_df[2]
    entity_labels = final_df[3]
    conll_lines = ""
    doc_start = "-DOCSTART- -X- -X- O\n" 
    conll_lines += "{}\n".format(doc_start)

    for tokens, pos, labels in zip(tokens,pos_labels, entity_labels):
        if "@@@@@" in tokens:
            sent_start = "" 
            
            conll_lines += "{}\n".format(sent_start)
        
        else:
            conll_lines += "{} {} {} {}\n".format(tokens,pos,pos,labels)
    return conll_lines

In [10]:
def export_txtfile(conll_lines, target_file): # helper to export the written CoNLL file
    with open(target_file,"w") as txtfile:
        for line in conll_lines:
            txtfile.write(line)
    txtfile.close()

In [11]:
def callsForConversion(final_txt_path, input_ann, output_conll):
    txt = read_input_ann(final_txt_path)
    main_list = generate_main_list(txt)
    final_list = map_lists(input_ann, main_list)
    conll_lines = export_to_CoNLL(final_list)
    export_txtfile(conll_lines, output_conll) # after this, you will have the fully ready CoNLL file

In [12]:
def main():

    final_txt_path = '/Users/arya/Desktop/AbhiramSinghvsC.D.Commachen&Orson16April,1996.txt'
    output_conll = '/Users/arya/Desktop/AbhiramSinghCoNLL.txt'
    input_ann = '/Users/arya/Desktop/AbhiramSinghvsC.D.Commachen&Orson16April,1996.ann'

    callsForConversion(final_txt_path, input_ann, output_conll)
    print("Data Converted to CoNLL Successfully")

if __name__ == "__main__":
    main()

Data Converted to CoNLL Successfully
